# Day 2: Bayesian Linear Regression with Uncertainty Quantification

## Learning Objectives
- Understand the Bayesian approach to linear regression
- Implement Bayesian linear regression from scratch
- Quantify uncertainty in predictions (epistemic vs aleatoric)
- Compare with frequentist approaches
- Apply to financial return prediction with confidence intervals

## Why Bayesian Regression in Finance?
- **Uncertainty Quantification**: Get probability distributions over predictions, not just point estimates
- **Risk Management**: Understand confidence in predictions for position sizing
- **Small Data**: Works better with limited data (common in finance)
- **Prior Knowledge**: Incorporate domain expertise through priors
- **Regularization**: Priors naturally regularize, preventing overfitting

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, multivariate_normal, invgamma
import yfinance as yf
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

---
## 1. Bayesian vs Frequentist Linear Regression

### Frequentist Approach (OLS)
$$y = X\beta + \epsilon, \quad \epsilon \sim N(0, \sigma^2)$$

Finds point estimate: $\hat{\beta} = (X^TX)^{-1}X^Ty$

### Bayesian Approach
Treats $\beta$ as a random variable with a prior distribution:

**Prior**: $p(\beta) = N(\mu_0, \Sigma_0)$

**Likelihood**: $p(y|X, \beta, \sigma^2) = N(X\beta, \sigma^2 I)$

**Posterior**: $p(\beta|y, X, \sigma^2) \propto p(y|X, \beta, \sigma^2) \cdot p(\beta)$

The posterior is also Gaussian (conjugate prior):
$$p(\beta|y, X, \sigma^2) = N(\mu_n, \Sigma_n)$$

Where:
- $\Sigma_n = (\Sigma_0^{-1} + \frac{1}{\sigma^2}X^TX)^{-1}$
- $\mu_n = \Sigma_n(\Sigma_0^{-1}\mu_0 + \frac{1}{\sigma^2}X^Ty)$

In [ ]:
class BayesianLinearRegression:
    """
    Bayesian Linear Regression with conjugate Gaussian prior.
    
    Assumes known noise variance sigma^2 (can be estimated from data).
    """
    
    def __init__(self, prior_mean=None, prior_cov=None, noise_var=1.0):
        """
        Initialize with prior parameters.
        
        Parameters:
        -----------
        prior_mean : array-like, shape (n_features,)
            Mean of prior distribution over weights
        prior_cov : array-like, shape (n_features, n_features)
            Covariance matrix of prior distribution
        noise_var : float
            Observation noise variance (sigma^2)
        """
        self.prior_mean = prior_mean
        self.prior_cov = prior_cov
        self.noise_var = noise_var
        self.posterior_mean = None
        self.posterior_cov = None
        self.n_features = None
        
    def _initialize_prior(self, n_features):
        """Initialize uninformative prior if not provided."""
        if self.prior_mean is None:
            self.prior_mean = np.zeros(n_features)
        if self.prior_cov is None:
            # Large variance = uninformative prior
            self.prior_cov = np.eye(n_features) * 100
            
    def fit(self, X, y):
        """
        Compute posterior distribution given data.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
        y : array-like, shape (n_samples,)
        """
        X = np.atleast_2d(X)
        y = np.atleast_1d(y)
        
        self.n_features = X.shape[1]
        self._initialize_prior(self.n_features)
        
        # Prior precision (inverse covariance)
        prior_precision = np.linalg.inv(self.prior_cov)
        
        # Posterior precision
        posterior_precision = prior_precision + (1/self.noise_var) * X.T @ X
        
        # Posterior covariance
        self.posterior_cov = np.linalg.inv(posterior_precision)
        
        # Posterior mean
        self.posterior_mean = self.posterior_cov @ (
            prior_precision @ self.prior_mean + 
            (1/self.noise_var) * X.T @ y
        )
        
        return self
    
    def predict(self, X, return_std=False):
        """
        Make predictions with uncertainty.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
        return_std : bool
            If True, return standard deviation of predictions
            
        Returns:
        --------
        y_mean : array, shape (n_samples,)
            Predictive mean
        y_std : array, shape (n_samples,)
            Predictive standard deviation (if return_std=True)
        """
        X = np.atleast_2d(X)
        
        # Predictive mean
        y_mean = X @ self.posterior_mean
        
        if return_std:
            # Predictive variance = epistemic + aleatoric
            # epistemic: X @ posterior_cov @ X.T (diagonal)
            # aleatoric: noise_var
            epistemic_var = np.sum(X @ self.posterior_cov * X, axis=1)
            total_var = epistemic_var + self.noise_var
            y_std = np.sqrt(total_var)
            return y_mean, y_std
        
        return y_mean
    
    def sample_posterior(self, n_samples=100):
        """
        Sample weight vectors from posterior distribution.
        
        Returns:
        --------
        samples : array, shape (n_samples, n_features)
        """
        return np.random.multivariate_normal(
            self.posterior_mean, 
            self.posterior_cov, 
            size=n_samples
        )
    
    def credible_interval(self, X, alpha=0.05):
        """
        Compute credible interval for predictions.
        
        Parameters:
        -----------
        X : array-like
        alpha : float
            Significance level (default: 0.05 for 95% CI)
            
        Returns:
        --------
        lower, upper : arrays
            Lower and upper bounds of credible interval
        """
        y_mean, y_std = self.predict(X, return_std=True)
        z = norm.ppf(1 - alpha/2)
        
        lower = y_mean - z * y_std
        upper = y_mean + z * y_std
        
        return lower, upper

print("BayesianLinearRegression class defined successfully!")

---
## 2. Synthetic Data Example: Understanding Uncertainty

In [ ]:
# Generate synthetic data
np.random.seed(42)
n_samples = 30
true_beta = np.array([2.0, -1.5])  # True weights
true_sigma = 0.5  # True noise std

# Generate X with a gap (to show uncertainty)
X_left = np.random.uniform(-3, -1, n_samples // 2)
X_right = np.random.uniform(1, 3, n_samples // 2)
X_train = np.concatenate([X_left, X_right])

# Design matrix with intercept and feature
X_design = np.column_stack([np.ones(len(X_train)), X_train])

# Generate y
y_train = X_design @ true_beta + np.random.normal(0, true_sigma, len(X_train))

print(f"Training samples: {len(X_train)}")
print(f"True coefficients: intercept={true_beta[0]:.2f}, slope={true_beta[1]:.2f}")
print(f"True noise std: {true_sigma}")

In [ ]:
# Fit Bayesian regression
blr = BayesianLinearRegression(
    prior_mean=np.zeros(2),      # Neutral prior
    prior_cov=np.eye(2) * 10,    # Moderately uninformative
    noise_var=true_sigma**2       # Known noise (in practice, estimate this)
)
blr.fit(X_design, y_train)

print("Posterior Distribution:")
print(f"Posterior mean: {blr.posterior_mean}")
print(f"Posterior std: {np.sqrt(np.diag(blr.posterior_cov))}")
print(f"\nTrue values: {true_beta}")

In [ ]:
# Visualize predictions with uncertainty
X_test = np.linspace(-4, 4, 200)
X_test_design = np.column_stack([np.ones(len(X_test)), X_test])

y_pred, y_std = blr.predict(X_test_design, return_std=True)
lower_95, upper_95 = blr.credible_interval(X_test_design, alpha=0.05)
lower_50, upper_50 = blr.credible_interval(X_test_design, alpha=0.50)

# Sample from posterior to show different plausible regression lines
posterior_samples = blr.sample_posterior(n_samples=50)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Predictions with credible intervals
ax = axes[0]
ax.scatter(X_train, y_train, c='black', s=50, zorder=5, label='Training data')
ax.plot(X_test, y_pred, 'b-', lw=2, label='Posterior mean')
ax.fill_between(X_test, lower_95, upper_95, alpha=0.2, color='blue', label='95% CI')
ax.fill_between(X_test, lower_50, upper_50, alpha=0.3, color='blue', label='50% CI')

# True line
y_true = X_test_design @ true_beta
ax.plot(X_test, y_true, 'r--', lw=2, label='True function')

ax.axvspan(-1, 1, alpha=0.1, color='red', label='No training data')
ax.set_xlabel('X')
ax.set_ylabel('y')
ax.set_title('Bayesian Regression: Uncertainty Quantification')
ax.legend(loc='upper right')

# Right: Posterior samples (multiple plausible lines)
ax = axes[1]
ax.scatter(X_train, y_train, c='black', s=50, zorder=5, label='Training data')

for i, beta_sample in enumerate(posterior_samples[:30]):
    y_sample = X_test_design @ beta_sample
    ax.plot(X_test, y_sample, 'b-', alpha=0.1, 
            label='Posterior samples' if i == 0 else '')

ax.plot(X_test, y_true, 'r--', lw=2, label='True function')
ax.axvspan(-1, 1, alpha=0.1, color='red')
ax.set_xlabel('X')
ax.set_ylabel('y')
ax.set_title('Posterior Samples: Plausible Regression Lines')
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

print("\n📊 Key Observation:")
print("Notice how uncertainty (credible interval width) INCREASES in the region")
print("where we have NO training data (-1 to 1). This is epistemic uncertainty!")

---
## 3. Types of Uncertainty

### Epistemic Uncertainty (Model Uncertainty)
- Comes from uncertainty in the model parameters
- **Reducible** with more data
- Captured by the posterior variance of $\beta$

### Aleatoric Uncertainty (Data/Observation Uncertainty)
- Comes from inherent noise in the data
- **Irreducible** - cannot be reduced with more data
- Captured by $\sigma^2$ (observation noise)

In [ ]:
def decompose_uncertainty(model, X):
    """
    Decompose predictive variance into epistemic and aleatoric components.
    """
    X = np.atleast_2d(X)
    
    # Epistemic: uncertainty about the weights
    epistemic_var = np.sum(X @ model.posterior_cov * X, axis=1)
    
    # Aleatoric: observation noise
    aleatoric_var = np.full(X.shape[0], model.noise_var)
    
    # Total
    total_var = epistemic_var + aleatoric_var
    
    return epistemic_var, aleatoric_var, total_var

# Decompose uncertainty
epistemic, aleatoric, total = decompose_uncertainty(blr, X_test_design)

fig, ax = plt.subplots(figsize=(12, 5))

ax.fill_between(X_test, 0, np.sqrt(aleatoric), alpha=0.5, 
                color='orange', label='Aleatoric (irreducible)')
ax.fill_between(X_test, np.sqrt(aleatoric), np.sqrt(total), alpha=0.5, 
                color='blue', label='Epistemic (reducible)')

ax.axvspan(-1, 1, alpha=0.1, color='red', label='No training data region')

# Mark training data regions
for x in X_train:
    ax.axvline(x, color='gray', alpha=0.2, lw=0.5)

ax.set_xlabel('X')
ax.set_ylabel('Predictive Std')
ax.set_title('Uncertainty Decomposition: Epistemic vs Aleatoric')
ax.legend()
plt.show()

print("\n📊 Interpretation:")
print("- Aleatoric uncertainty is CONSTANT (inherent noise in observations)")
print("- Epistemic uncertainty VARIES with distance from training data")
print("- In the gap region, epistemic uncertainty dominates!")

---
## 4. Effect of Data Size on Uncertainty

In [ ]:
def fit_with_n_samples(n):
    """Fit Bayesian regression with n samples."""
    np.random.seed(42)
    X = np.random.uniform(-3, 3, n)
    X_design = np.column_stack([np.ones(n), X])
    y = X_design @ true_beta + np.random.normal(0, true_sigma, n)
    
    model = BayesianLinearRegression(
        prior_mean=np.zeros(2),
        prior_cov=np.eye(2) * 10,
        noise_var=true_sigma**2
    )
    model.fit(X_design, y)
    return model

# Fit with different sample sizes
sample_sizes = [5, 20, 100, 500]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

X_plot = np.linspace(-4, 4, 200)
X_plot_design = np.column_stack([np.ones(len(X_plot)), X_plot])
y_true_plot = X_plot_design @ true_beta

for i, n in enumerate(sample_sizes):
    model = fit_with_n_samples(n)
    y_pred, y_std = model.predict(X_plot_design, return_std=True)
    
    ax = axes[i]
    ax.plot(X_plot, y_pred, 'b-', lw=2, label='Posterior mean')
    ax.fill_between(X_plot, y_pred - 2*y_std, y_pred + 2*y_std, 
                    alpha=0.3, color='blue', label='95% CI')
    ax.plot(X_plot, y_true_plot, 'r--', lw=2, label='True function')
    
    # Show posterior std on weights
    weight_std = np.sqrt(np.diag(model.posterior_cov))
    ax.set_title(f'n={n} samples\n'
                 f'Weight uncertainty: β₀±{weight_std[0]:.3f}, β₁±{weight_std[1]:.3f}')
    ax.set_xlabel('X')
    ax.set_ylabel('y')
    ax.legend(loc='upper right')
    ax.set_ylim(-10, 10)

plt.suptitle('Effect of Sample Size on Epistemic Uncertainty', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("\n📊 Key Insight:")
print("As n increases, epistemic uncertainty (credible interval) SHRINKS")
print("Eventually converging to just the aleatoric uncertainty (noise).")

---
## 5. Financial Application: Factor Return Prediction

In [ ]:
# Download market data
print("Downloading financial data...")

tickers = ['SPY', 'TLT', 'GLD', 'VIX']  # Market, Bonds, Gold, Volatility
data = yf.download(tickers, start='2018-01-01', end='2024-01-01', progress=False)['Adj Close']

# Calculate returns
returns = data.pct_change().dropna()

# Target: SPY returns
# Features: lagged returns of all assets
df = pd.DataFrame()
df['target'] = returns['SPY']

# Lagged features (1-day lag)
for ticker in tickers:
    df[f'{ticker}_lag1'] = returns[ticker].shift(1)
    df[f'{ticker}_lag2'] = returns[ticker].shift(2)
    df[f'{ticker}_lag5'] = returns[ticker].shift(5)  # Weekly lag

df = df.dropna()
print(f"\nDataset shape: {df.shape}")
print(f"Date range: {df.index[0].date()} to {df.index[-1].date()}")
df.head()

In [ ]:
# Prepare features and target
feature_cols = [c for c in df.columns if c != 'target']
X = df[feature_cols].values
y = df['target'].values

# Split: use last year as test
split_idx = -252  # ~1 year of trading days
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]
dates_test = df.index[split_idx:]

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Add intercept
X_train_design = np.column_stack([np.ones(len(X_train_scaled)), X_train_scaled])
X_test_design = np.column_stack([np.ones(len(X_test_scaled)), X_test_scaled])

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Features: {len(feature_cols)}")

In [ ]:
# Estimate noise variance from training data residuals (using OLS first)
ols = LinearRegression(fit_intercept=False)
ols.fit(X_train_design, y_train)
residuals = y_train - ols.predict(X_train_design)
estimated_noise_var = np.var(residuals)

print(f"Estimated noise std: {np.sqrt(estimated_noise_var):.4f}")
print(f"(Daily SPY volatility: {np.std(y_train):.4f})")

# Fit Bayesian regression
n_features = X_train_design.shape[1]

# Informative prior: we expect small coefficients for return prediction
prior_mean = np.zeros(n_features)
prior_cov = np.eye(n_features) * 0.01  # Regularizing prior (like Ridge)

blr_finance = BayesianLinearRegression(
    prior_mean=prior_mean,
    prior_cov=prior_cov,
    noise_var=estimated_noise_var
)
blr_finance.fit(X_train_design, y_train)

print("\nBayesian regression fitted!")

In [ ]:
# Analyze coefficient uncertainty
feature_names = ['intercept'] + feature_cols
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Posterior Mean': blr_finance.posterior_mean,
    'Posterior Std': np.sqrt(np.diag(blr_finance.posterior_cov)),
    'OLS Estimate': ols.coef_
})
coef_df['95% CI Lower'] = coef_df['Posterior Mean'] - 1.96 * coef_df['Posterior Std']
coef_df['95% CI Upper'] = coef_df['Posterior Mean'] + 1.96 * coef_df['Posterior Std']
coef_df['Significant'] = (coef_df['95% CI Lower'] > 0) | (coef_df['95% CI Upper'] < 0)

print("\nCoefficient Posterior Distributions:")
print(coef_df.to_string(index=False))

In [ ]:
# Visualize coefficient uncertainty
fig, ax = plt.subplots(figsize=(12, 6))

y_pos = np.arange(len(feature_names))
colors = ['green' if sig else 'gray' for sig in coef_df['Significant']]

ax.barh(y_pos, coef_df['Posterior Mean'], xerr=1.96*coef_df['Posterior Std'],
        color=colors, alpha=0.7, capsize=3)
ax.axvline(0, color='black', linestyle='--', lw=1)
ax.set_yticks(y_pos)
ax.set_yticklabels(feature_names)
ax.set_xlabel('Coefficient Value')
ax.set_title('Bayesian Regression Coefficients with 95% Credible Intervals\n'
             '(Green = CI excludes 0, Gray = CI includes 0)')
plt.tight_layout()
plt.show()

significant_features = coef_df[coef_df['Significant']]['Feature'].tolist()
print(f"\n📊 Features with 95% credible intervals excluding 0:")
print(f"   {significant_features if significant_features else 'None - high uncertainty!'}")

In [ ]:
# Make predictions with uncertainty
y_pred_test, y_std_test = blr_finance.predict(X_test_design, return_std=True)
lower_95, upper_95 = blr_finance.credible_interval(X_test_design, alpha=0.05)

# Create results DataFrame
results = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred_test,
    'Std': y_std_test,
    'Lower_95': lower_95,
    'Upper_95': upper_95
}, index=dates_test)

# Check calibration: % of actuals within 95% CI
coverage = np.mean((y_test >= lower_95) & (y_test <= upper_95))
print(f"95% Credible Interval Coverage: {coverage:.1%}")
print(f"(Should be close to 95% for well-calibrated uncertainty)")

# RMSE comparison
ols_pred = ols.predict(X_test_design)
rmse_bayesian = np.sqrt(np.mean((y_test - y_pred_test)**2))
rmse_ols = np.sqrt(np.mean((y_test - ols_pred)**2))
print(f"\nRMSE Bayesian: {rmse_bayesian:.6f}")
print(f"RMSE OLS: {rmse_ols:.6f}")

In [ ]:
# Visualize predictions with uncertainty bands
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Top: Time series of predictions
ax = axes[0]
ax.plot(results.index, results['Actual'], 'k-', alpha=0.7, lw=0.8, label='Actual')
ax.plot(results.index, results['Predicted'], 'b-', alpha=0.7, lw=1, label='Predicted')
ax.fill_between(results.index, results['Lower_95'], results['Upper_95'], 
                alpha=0.3, color='blue', label='95% CI')
ax.set_ylabel('Daily Return')
ax.set_title('SPY Return Prediction with Bayesian Uncertainty')
ax.legend(loc='upper right')
ax.axhline(0, color='gray', linestyle='--', alpha=0.5)

# Bottom: Prediction errors vs uncertainty
ax = axes[1]
errors = np.abs(results['Actual'] - results['Predicted'])
ax.scatter(results['Std'], errors, alpha=0.5, s=20)

# Add regression line
z = np.polyfit(results['Std'], errors, 1)
p = np.poly1d(z)
x_line = np.linspace(results['Std'].min(), results['Std'].max(), 100)
ax.plot(x_line, p(x_line), 'r--', lw=2, label=f'Trend: y={z[0]:.2f}x+{z[1]:.4f}')

ax.set_xlabel('Predicted Uncertainty (Std)')
ax.set_ylabel('Absolute Prediction Error')
ax.set_title('Error vs Uncertainty: Are Uncertain Predictions Less Accurate?')
ax.legend()

plt.tight_layout()
plt.show()

# Correlation between uncertainty and error
corr = np.corrcoef(results['Std'], errors)[0, 1]
print(f"\n📊 Correlation between uncertainty and |error|: {corr:.3f}")
print("Positive correlation means higher uncertainty → larger errors (well-calibrated!)")

---
## 6. Using Uncertainty for Position Sizing

In [ ]:
def uncertainty_weighted_strategy(predictions, uncertainties, risk_aversion=1.0):
    """
    Scale positions by prediction confidence.
    
    Position = prediction / (uncertainty * risk_aversion)
    
    High confidence (low uncertainty) → larger positions
    Low confidence (high uncertainty) → smaller positions
    """
    # Information ratio based sizing
    positions = predictions / (uncertainties * risk_aversion)
    
    # Normalize to reasonable leverage
    positions = np.clip(positions, -2, 2)  # Max 2x leverage
    
    return positions

# Compare strategies
# 1. Equal weight: always long/short based on prediction sign
positions_equal = np.sign(y_pred_test)

# 2. Uncertainty weighted
positions_uncertainty = uncertainty_weighted_strategy(y_pred_test, y_std_test, risk_aversion=1.0)

# Calculate returns
returns_equal = positions_equal * y_test
returns_uncertainty = positions_uncertainty * y_test
returns_buy_hold = y_test

# Calculate cumulative returns
cum_equal = (1 + returns_equal).cumprod()
cum_uncertainty = (1 + returns_uncertainty).cumprod()
cum_buy_hold = (1 + returns_buy_hold).cumprod()

# Statistics
def calc_stats(returns):
    return {
        'Total Return': (1 + returns).prod() - 1,
        'Ann. Volatility': returns.std() * np.sqrt(252),
        'Sharpe': returns.mean() / returns.std() * np.sqrt(252),
        'Max Drawdown': (pd.Series((1 + returns).cumprod()).cummax() - 
                        pd.Series((1 + returns).cumprod())).max()
    }

stats_df = pd.DataFrame({
    'Buy & Hold': calc_stats(returns_buy_hold),
    'Equal Weight': calc_stats(returns_equal),
    'Uncertainty Weighted': calc_stats(returns_uncertainty)
}).T

print("\nStrategy Comparison:")
print(stats_df.round(4))

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Cumulative returns
ax = axes[0]
ax.plot(dates_test, cum_buy_hold, label='Buy & Hold', lw=2)
ax.plot(dates_test, cum_equal, label='Equal Weight (sign of prediction)', lw=2)
ax.plot(dates_test, cum_uncertainty, label='Uncertainty Weighted', lw=2)
ax.set_ylabel('Cumulative Return')
ax.set_title('Strategy Comparison: Leveraging Uncertainty')
ax.legend()
ax.axhline(1, color='gray', linestyle='--', alpha=0.5)

# Position sizes over time
ax = axes[1]
ax.fill_between(dates_test, 0, positions_uncertainty, alpha=0.5, label='Uncertainty Weighted Position')
ax.axhline(0, color='black', linestyle='-', lw=0.5)
ax.set_ylabel('Position Size')
ax.set_xlabel('Date')
ax.set_title('Position Sizing Based on Prediction Confidence')
ax.legend()

plt.tight_layout()
plt.show()

print("\n📊 Key Insight:")
print("Uncertainty-weighted positions scale down during high uncertainty periods,")
print("potentially reducing drawdowns and improving risk-adjusted returns.")

---
## 7. Scikit-learn's BayesianRidge Comparison

In [ ]:
# Scikit-learn's Bayesian Ridge automatically estimates noise variance
from sklearn.linear_model import BayesianRidge

br_sklearn = BayesianRidge(
    alpha_1=1e-6, alpha_2=1e-6,  # Gamma prior on alpha (precision of weights)
    lambda_1=1e-6, lambda_2=1e-6, # Gamma prior on lambda (precision of noise)
    fit_intercept=True
)
br_sklearn.fit(X_train_scaled, y_train)

# Get predictions with uncertainty
y_pred_sklearn, y_std_sklearn = br_sklearn.predict(X_test_scaled, return_std=True)

print("Scikit-learn BayesianRidge:")
print(f"Estimated noise precision (alpha): {br_sklearn.alpha_:.6f}")
print(f"Estimated weight precision (lambda): {br_sklearn.lambda_:.6f}")

# Compare with our implementation
rmse_sklearn = np.sqrt(np.mean((y_test - y_pred_sklearn)**2))
print(f"\nRMSE sklearn: {rmse_sklearn:.6f}")
print(f"RMSE our impl: {rmse_bayesian:.6f}")

In [ ]:
# Compare coefficient estimates
coef_comparison = pd.DataFrame({
    'Feature': feature_cols,
    'Our Bayesian': blr_finance.posterior_mean[1:],  # Skip intercept
    'Sklearn Bayesian': br_sklearn.coef_,
    'OLS': ols.coef_[1:]
})

print("\nCoefficient Comparison:")
print(coef_comparison.round(6).to_string(index=False))

---
## 8. Sequential Bayesian Updating (Online Learning)

In [ ]:
class OnlineBayesianRegression:
    """
    Bayesian regression with sequential updates.
    Uses the posterior from previous observations as prior for new ones.
    """
    
    def __init__(self, n_features, prior_mean=None, prior_cov=None, noise_var=1.0):
        self.n_features = n_features
        self.noise_var = noise_var
        
        # Initialize prior
        self.mean = prior_mean if prior_mean is not None else np.zeros(n_features)
        self.cov = prior_cov if prior_cov is not None else np.eye(n_features) * 100
        
        self.history = {'mean': [self.mean.copy()], 'cov': [self.cov.copy()]}
        
    def update(self, x, y):
        """
        Update posterior with single observation.
        
        Parameters:
        -----------
        x : array, shape (n_features,)
        y : float
        """
        x = np.atleast_1d(x).reshape(-1, 1)  # Column vector
        
        # Kalman-like update
        # S = x.T @ cov @ x + noise_var  (scalar)
        S = x.T @ self.cov @ x + self.noise_var
        
        # Kalman gain
        K = self.cov @ x / S
        
        # Update mean
        innovation = y - x.T @ self.mean
        self.mean = self.mean + (K * innovation).flatten()
        
        # Update covariance
        self.cov = self.cov - K @ x.T @ self.cov
        
        # Store history
        self.history['mean'].append(self.mean.copy())
        self.history['cov'].append(self.cov.copy())
        
    def predict(self, X, return_std=False):
        """Predict using current posterior."""
        X = np.atleast_2d(X)
        y_mean = X @ self.mean
        
        if return_std:
            var = np.sum(X @ self.cov * X, axis=1) + self.noise_var
            return y_mean, np.sqrt(var)
        return y_mean

print("OnlineBayesianRegression class defined!")

In [ ]:
# Demonstrate online learning: watch uncertainty decrease as data arrives
np.random.seed(42)

# Simple 1D example with intercept
n_online = 100
X_online = np.random.uniform(-3, 3, n_online)
X_online_design = np.column_stack([np.ones(n_online), X_online])
y_online = X_online_design @ true_beta + np.random.normal(0, 0.5, n_online)

# Initialize online learner
online_blr = OnlineBayesianRegression(
    n_features=2,
    prior_mean=np.zeros(2),
    prior_cov=np.eye(2) * 10,
    noise_var=0.25
)

# Process observations one at a time
for i in range(n_online):
    online_blr.update(X_online_design[i], y_online[i])

print(f"After {n_online} sequential updates:")
print(f"Posterior mean: {online_blr.mean}")
print(f"True values: {true_beta}")
print(f"Posterior std: {np.sqrt(np.diag(online_blr.cov))}")

In [ ]:
# Visualize how uncertainty evolves over time
means_history = np.array(online_blr.history['mean'])
stds_history = np.array([np.sqrt(np.diag(c)) for c in online_blr.history['cov']])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for i, (name, true_val) in enumerate([('Intercept (β₀)', true_beta[0]), ('Slope (β₁)', true_beta[1])]):
    ax = axes[i]
    n_obs = np.arange(len(means_history))
    
    ax.plot(n_obs, means_history[:, i], 'b-', lw=2, label='Posterior mean')
    ax.fill_between(n_obs, 
                    means_history[:, i] - 2*stds_history[:, i],
                    means_history[:, i] + 2*stds_history[:, i],
                    alpha=0.3, color='blue', label='95% CI')
    ax.axhline(true_val, color='red', linestyle='--', lw=2, label=f'True value: {true_val}')
    
    ax.set_xlabel('Number of Observations')
    ax.set_ylabel('Parameter Value')
    ax.set_title(f'{name} - Sequential Bayesian Learning')
    ax.legend()

plt.tight_layout()
plt.show()

print("\n📊 Key Observation:")
print("The credible interval SHRINKS as more data arrives (epistemic uncertainty decreases).")
print("The posterior mean converges to the true value!")

---
## 9. Calibration Analysis

In [ ]:
def calibration_analysis(y_true, y_pred, y_std, n_bins=10):
    """
    Check if predicted uncertainties are calibrated.
    
    For each confidence level, check if the actual coverage matches.
    """
    confidence_levels = np.linspace(0.1, 0.99, 20)
    actual_coverage = []
    
    for conf in confidence_levels:
        z = norm.ppf((1 + conf) / 2)
        lower = y_pred - z * y_std
        upper = y_pred + z * y_std
        coverage = np.mean((y_true >= lower) & (y_true <= upper))
        actual_coverage.append(coverage)
    
    return confidence_levels, np.array(actual_coverage)

# Check calibration for our Bayesian model
conf_levels, actual_cov = calibration_analysis(y_test, y_pred_test, y_std_test)

fig, ax = plt.subplots(figsize=(8, 8))

ax.plot([0, 1], [0, 1], 'k--', lw=2, label='Perfect calibration')
ax.plot(conf_levels, actual_cov, 'bo-', lw=2, markersize=6, label='Bayesian Regression')

ax.set_xlabel('Expected Coverage (Confidence Level)')
ax.set_ylabel('Actual Coverage')
ax.set_title('Calibration Plot: Expected vs Actual Coverage')
ax.legend()
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

# Calculate calibration error
calibration_error = np.mean(np.abs(conf_levels - actual_cov))
print(f"\nMean Calibration Error: {calibration_error:.3f}")
print("(Lower is better, 0 = perfectly calibrated)")

---
## 10. Summary & Key Takeaways

### What We Learned:

1. **Bayesian vs Frequentist**:
   - Frequentist: Point estimates only
   - Bayesian: Full probability distributions over predictions

2. **Uncertainty Types**:
   - **Epistemic**: Model uncertainty, reducible with more data
   - **Aleatoric**: Observation noise, irreducible

3. **Financial Applications**:
   - Uncertainty-based position sizing
   - Confidence intervals for risk management
   - Online learning for adaptive models

4. **Practical Benefits**:
   - Built-in regularization through priors
   - Works well with limited data
   - Principled uncertainty quantification

### Key Formulas:

**Posterior Distribution**:
$$\Sigma_n = (\Sigma_0^{-1} + \frac{1}{\sigma^2}X^TX)^{-1}$$
$$\mu_n = \Sigma_n(\Sigma_0^{-1}\mu_0 + \frac{1}{\sigma^2}X^Ty)$$

**Predictive Distribution**:
$$p(y_*|x_*, X, y) = N(x_*^T\mu_n, x_*^T\Sigma_n x_* + \sigma^2)$$

In [ ]:
# Quick reference implementation
print("=" * 60)
print("QUICK REFERENCE: Bayesian Linear Regression")
print("=" * 60)

quick_ref = """
# 1. Basic Usage (our implementation)
blr = BayesianLinearRegression(
    prior_mean=np.zeros(n_features),  # Prior belief about weights
    prior_cov=np.eye(n_features) * 10,  # Prior uncertainty
    noise_var=estimated_noise_var  # Observation noise
)
blr.fit(X_train, y_train)
y_pred, y_std = blr.predict(X_test, return_std=True)

# 2. Scikit-learn
from sklearn.linear_model import BayesianRidge
br = BayesianRidge()
br.fit(X_train, y_train)
y_pred, y_std = br.predict(X_test, return_std=True)

# 3. Credible Intervals
z = 1.96  # For 95% CI
lower = y_pred - z * y_std
upper = y_pred + z * y_std

# 4. Position Sizing with Uncertainty
positions = predictions / (uncertainties * risk_aversion)
"""
print(quick_ref)

---
## Exercises

1. **Prior Sensitivity**: How do different priors affect the posterior? Try informative vs uninformative priors.

2. **Unknown Variance**: Implement Bayesian regression with unknown noise variance (conjugate prior: Normal-Inverse-Gamma).

3. **Feature Selection**: Use the credible intervals to perform Bayesian feature selection.

4. **Rolling Bayesian**: Implement a rolling window Bayesian regression for adaptive return prediction.

5. **Compare with MCMC**: Use PyMC or Stan to implement Bayesian regression and compare with the analytical solution.